This code creates LDPC codewords from base matrices following the 5G standard. It takes base matrices from text files
to create the LDPC H matrix and encode messages. The original MATLAB code and video lectures can be found [here](https://nptel.ac.in/courses/108106137/)

In [1]:
%matplotlib notebook
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

In [2]:
B = np.loadtxt('base_matrices/NR_1_2_20.txt') # last two digits in the filename indicate the expansion factor to be used
z = 20
k = (B.shape[1] - B.shape[0]) * z
n = B.shape[1] * z
B = B.astype(int)
print("Base matrix shape: {}".format(B.shape))
print("Number of block messages: {}".format(B.shape[1] - B.shape[0]))
print("Number of message bits k: {}".format(k))
print("Total number of bits n: {}".format(n))
print("Code rate r: {:.3f}".format(k / n))

Base matrix shape: (46, 68)
Number of block messages: 22
Number of message bits k: 440
Total number of bits n: 1360
Code rate r: 0.324


Set a specified code rate:

In [3]:
r = 0.5
k = B.shape[1] - B.shape[0]
n = int(k / r)
B = B[:k, :n]

Visualise the base matrix:

In [4]:
# plt.matshow(B, fignum=10)
# plt.gca().set_aspect('auto')
# plt.show()

Expand the base matrix to create the LDPC H matrix:

In [5]:
row = []
col = []
ef = z # expansion factor
for i in range(B.shape[0]):
    row = []
    for j in range(B.shape[1]):
        k = B[i, j]
        if k == -1:
            mat = np.zeros((ef, ef))
        elif k == 0:
            mat = np.eye(ef, ef)
        else:
            mat = np.eye(ef, ef)
            mat = np.roll(mat, k, axis=1)
        row.append(mat)
        mat = np.concatenate(row, axis=1)
    col.append(mat)
H = np.concatenate(col, axis=0).astype(int)
print("Final shape of H matrix: {}".format(H.shape))

Final shape of H matrix: (440, 880)


Visualise the final H matrix:

In [6]:
# plt.matshow(H, fignum=10)
# plt.gca().set_aspect('auto')
# plt.show()

Encoding messages to get the LDPC codewords:

In [7]:
def mul_shft(x, k):
    # multiplication by the shifted identity matrix
    # x is the input block and k is the shift
    if k == -1:
        return np.zeros(x.shape[0])
    else:
        return np.array(list(x[k:]) + list(x[:k]))

In [8]:
def check_valid_codeword(H, c):
    # modulo 2 matrix multiplication between H and codeword
    # if all zeros then c is a valid codeword
    return (np.matmul(H, c) % 2).sum()

In [9]:
def get_codeword(B, z, msg):
    # encode the message bits msg
    m, n = B.shape
    cword = np.zeros(n*z)
    cword[:(n-m)*z] = msg

    # double-diagonal encoding
    temp = np.zeros(z)
    for i in range(4):
        for j in range(n - m):
            temp = (temp + mul_shft(msg[j*z:(j + 1)*z], B[i, j])) % 2

    # find the first parity bit p1        
    if B[1, n-m] == -1:
        p1_sh = B[2, n-m];
    else:
        p1_sh = B[1, n-m];

    cword[(n-m)*z:(n-m+1)*z] = mul_shft(temp, z-p1_sh)

    # find p2, p3, p4
    for i in range(3):
        temp = np.zeros(z)
        for j in range(n-m+i+1):
            temp = (temp + mul_shft(cword[j*z:(j + 1)*z], B[i,j])) % 2
        cword[(n-m+i+1)*z:(n-m+i+2)*z] = temp
        
    # find the remaining parity bits
    for i in range(4, m):
        temp = np.zeros(z)
        for j in range(n-m+4):
            temp = (temp + mul_shft(cword[j*z:(j + 1)*z], B[i,j])) % 2       
        cword[(n-m+i)*z:(n-m+i+1)*z] = temp
    return cword.astype(int)

Now we create random messages and encode them:

In [10]:
codeword_list = []
for i in range(10000):
    msg = np.random.randint(0, 2, H.shape[0])
    codeword = get_codeword(B, z, msg)
    if check_valid_codeword(H, codeword) == 0:
        codeword_list.append(list(codeword))
    else:
        print("Bad codeword found")

Store all the codewords:

In [11]:
# file_name = 'NR_1_0_2_codewords'
file_name = 'NR_1_2_20_codewords'
df = pd.DataFrame(codeword_list)
df.to_csv('{}.csv'.format(file_name), sep=',', header=False, index=False)

This part is used to create the even parity check factors that is used in infer.net: 

In [12]:
# for i, col in enumerate(H):
#     positions = np.where(col == 1)[0]
#     if len(positions) > 0:
#         factor = " != ".join("v[{0}]".format(n) for n in positions)
#         print("Variable.ConstrainEqual({}, Variable.Bernoulli(0));".format(factor))

Side note: the relationship between SNR and precision:

SNR = $10 \times \text{log}_{10}(\frac{1}{var})$ (dB) = $10 \times \text{log}_{10}(\gamma)$, where $\gamma$ is the precision.

$\gamma = 10^{(\frac{SNR}{10})}$ (precision) or 
$var = \sigma^2 = 10^{(\frac{-SNR}{10})}$ (variance) or
$\sigma = 10^{(\frac{-SNR}{20})}$ (standard deviation)

